In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 3.5 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

In [4]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
chmod u+rx ipopt

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  libgfortran5
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 589 kB of archives.
After this operation, 2,660 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libgfortran5 amd64 8.4.0-1ubuntu1~18.04 [589 kB]
Fetched 589 kB in 1s (846 kB/s)
Selecting previously unselected package libgfortran5:amd64.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../libgfortran5_8.4.0-1ubuntu1~18.04_amd64.deb ...
Unpacking libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Setting up libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [5]:
import pandas as pd

In [6]:
data_csvfile = pd.read_csv('lab6_ex3.csv')

In [7]:
model = ConcreteModel()

In [8]:
N = len(data_csvfile.index)

In [9]:
col_indices = range(N)

In [10]:
model.r = Var(col_indices)
model.b0 = Var(domain = NonNegativeReals)
model.b1 = Var()
model.b2 = Var()

In [11]:
model.objective = Objective(expr = sum(model.r[i]**2 for i in col_indices), sense = minimize)

In [12]:
model.constraints = ConstraintList()

In [13]:
for i in col_indices:
  model.constraints.add(data_csvfile['SP'][i] - (model.b0 + model.b1*data_csvfile['Lot Size'][i] + model.b2*data_csvfile['Elevation'][i]) == model.r[i])

In [14]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    r_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

4 Var Declarations
    b0 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    b1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    b2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    r : Size=6, Index=r_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  

In [15]:
opt_ipopt = SolverFactory('ipopt')
result = opt_ipopt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('r[',i,'] =', model.r[i].value)  
print('b0 = ', model.b0.value)
print('b1 = ', model.b1.value)
print('b2 = ', model.b2.value)  


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 9
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02468276023864746
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  4291134028.807458

Decision Variables
r[ 0 ] = 48762.444397373445
r[ 1 ] = 14058.726964626147
r[ 2 ] = -28366.606651133246
r[ 3 ] = -26533.838169879265
r[ 4 ] = -13331.391654310524
r[ 5 ] = 5410.665113323442
b0 =  104904.41643719551
b1 =  38.70207583139165
b2 =  -193.32238932429573


#Ans 2 :

Solver Status : Ok

Solver Termination Condition : Optimal

Objective Value : 4291134028.807

Values of decision variables :

$r_1=48762.44\\
r_2=14058.73\\
r_3=-28366.61\\
r_4=-26533.84\\
r_5=-13331.39\\
r_6=5410.67$

Values of constants, $b_0,b_1,b_2$

$b_0 = 104904.42\\
b_1 = 38.70\\
b_2 = -193.32$

**Maximum value of residual from linear model with absolute terms is : -54272.727**

**Maximum value of residual from this model with absolute terms is : 48762.44**

For comparison we take the absolute values, and see the difference, i.e. 5510.287

There is a difference between them as $R^2$ gives more weight to larger errors as it squares the errors.

R^2 will have lower maximum value than linear model ith absolute errors.
